In [ ]:
import tkinter as tk
from tkinter import messagebox, simpledialog, ttk

class CanteenManagement:
    def __init__(self, root):
        self.root = root
        self.root.title("Canteen Management System")
        self.root.geometry("500x600")
        self.root.configure(bg="#f4f4f4")

        # Predefined menu items
        self.menu = {
            "Burger": 50,
            "Pizza": 120,
            "Pasta": 80,
            "Sandwich": 40,
            "Coffee": 30,
            "Tea": 20,
            "French Fries": 60
        }
        self.stock = {item: 0 for item in self.menu}  # Initialize stock for each item
        self.cart = {}

        tk.Label(root, text="Canteen Menu", font=("Arial", 16, "bold"), bg="#ffcc00", fg="black", padx=10, pady=5).pack(fill=tk.X)
        self.menu_listbox = tk.Listbox(root, width=50, height=10, font=("Arial", 12))
        self.menu_listbox.pack(pady=10)
        self.display_menu()

        tk.Label(root, text="Select User Type", font=("Arial", 14, "bold"), bg="#ff5733", fg="white", padx=10, pady=5).pack(fill=tk.X)
        tk.Button(root, text="Owner", font=("Arial", 12), bg="#33cc33", fg="white", command=self.owner_login).pack(pady=5)
        tk.Button(root, text="Customer", font=("Arial", 12), bg="#3399ff", fg="white", command=self.customer_interface).pack(pady=5)

    def display_menu(self):
        self.menu_listbox.delete(0, tk.END)
        for item, price in self.menu.items():
            stock = self.stock.get(item, 0)
            self.menu_listbox.insert(tk.END, f"{item}: ₹{price} | Stock: {stock}")

    def owner_login(self):
        password = simpledialog.askstring("Owner Login", "Enter Owner Password:", show='*')
        if password == "admin123":
            self.owner_interface()
        else:
            messagebox.showerror("Access Denied", "Incorrect password!")

    def owner_interface(self):
        owner_window = tk.Toplevel(self.root)
        owner_window.title("Owner Panel")
        owner_window.geometry("400x400")
        owner_window.configure(bg="#e6e6fa")

        tk.Label(owner_window, text="Manage Menu", font=("Arial", 14, "bold"), bg="#ff5733", fg="white").pack(fill=tk.X)
        tk.Button(owner_window, text="Add Item", font=("Arial", 12), bg="#33cc33", fg="white", command=self.add_item).pack(pady=5)
        tk.Button(owner_window, text="Remove Item", font=("Arial", 12), bg="#cc3333", fg="white", command=self.remove_item).pack(pady=5)
        tk.Button(owner_window, text="Modify Item", font=("Arial", 12), bg="#ffcc00", fg="black", command=self.modify_item).pack(pady=5)
        tk.Button(owner_window, text="Set Stock", font=("Arial", 12), bg="#3399ff", fg="white", command=self.set_stock).pack(pady=5)

    def set_stock(self):
        for item in self.menu:
            qty = simpledialog.askinteger("Set Stock", f"Enter stock for {item}:")
            if qty is not None:
                self.stock[item] = qty
        self.display_menu()

    def add_item(self):
        item = simpledialog.askstring("Add Item", "Enter item name:")
        price = simpledialog.askinteger("Add Item", "Enter price:")
        if item and price:
            self.menu[item] = price
            self.stock[item] = 0
            self.display_menu()
            messagebox.showinfo("Success", f"{item} added successfully!")

    def remove_item(self):
        item = simpledialog.askstring("Remove Item", "Enter item name to remove:")
        if item in self.menu:
            del self.menu[item]
            del self.stock[item]
            self.display_menu()
            messagebox.showinfo("Success", f"{item} removed successfully!")
        else:
            messagebox.showerror("Error", "Item not found!")

    def modify_item(self):
        item = simpledialog.askstring("Modify Item", "Enter item name to modify:")
        if item in self.menu:
            new_price = simpledialog.askinteger("Modify Item", "Enter new price:")
            self.menu[item] = new_price
            self.display_menu()
            messagebox.showinfo("Success", f"{item} modified successfully!")
        else:
            messagebox.showerror("Error", "Item not found!")

    def customer_interface(self):
        self.cart = {}
        self.order_more()

    def order_more(self):
        customer_window = tk.Toplevel(self.root)
        customer_window.title("Customer Ordering")
        customer_window.geometry("400x400")
        customer_window.configure(bg="#e6e6fa")

        tk.Label(customer_window, text="Select Item", font=("Arial", 14, "bold"), bg="#ffcc00").pack(fill=tk.X)
        self.item_var = tk.StringVar()
        self.item_dropdown = ttk.Combobox(customer_window, textvariable=self.item_var, values=list(self.menu.keys()), font=("Arial", 12))
        self.item_dropdown.pack(pady=5)

        tk.Label(customer_window, text="Enter Quantity", font=("Arial", 14, "bold"), bg="#ffcc00").pack(fill=tk.X)
        self.qty_var = tk.IntVar()
        self.qty_entry = tk.Entry(customer_window, textvariable=self.qty_var, font=("Arial", 12))
        self.qty_entry.pack(pady=5)

        def add_and_ask():
            item = self.item_var.get()
            qty = self.qty_var.get()

            if item and qty > 0:
                if self.stock[item] >= qty:
                    self.cart[item] = self.cart.get(item, 0) + qty
                    self.stock[item] -= qty
                    messagebox.showinfo("Cart Update", f"Added {qty} x {item} to cart.")
                    self.display_menu()
                else:
                    messagebox.showwarning("Out of Stock", f"Only {self.stock[item]} {item} left in stock.")
            else:
                messagebox.showwarning("Invalid Input", "Please select an item and enter a valid quantity.")

            more = messagebox.askyesno("Continue", "Do you want to add more items?")
            customer_window.destroy()
            if more:
                self.order_more()
            else:
                self.checkout()

        tk.Button(customer_window, text="Add to Cart", font=("Arial", 12), bg="#33cc33", fg="white", command=add_and_ask).pack(pady=5)

    def checkout(self):
        if not self.cart:
            messagebox.showwarning("Empty Cart", "Your cart is empty!")
            return

        total = sum(self.menu[item] * qty for item, qty in self.cart.items())
        order_summary = "\n".join([f"{item} x {qty} = ₹{self.menu[item] * qty}" for item, qty in self.cart.items()])
        messagebox.showinfo("Order Summary", f"Your Order:\n{order_summary}\n\nTotal Amount to Pay: ₹{total}")
        self.cart.clear()

if __name__ == "__main__":
    root = tk.Tk()
    app = CanteenManagement(root)
    root.mainloop()